# Trabalho 2 - Computação Gráfica SCC0250

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [196]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

### Inicializando janela

In [197]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1600
largura = 1200
window = glfw.create_window(largura, altura, "Malhas e Texturas", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base nas transformações Model, View e Projection.

In [198]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        varying vec2 out_texture;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [199]:
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [200]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [201]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [202]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [203]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [204]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [205]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.

In [206]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])


        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model



In [207]:
glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
glEnable( GL_BLEND )
glBlendFunc( GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA )
glEnable(GL_LINE_SMOOTH)
glEnable(GL_TEXTURE_2D)
qtd_texturas = 4
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    print(img_textura,img.mode)
    img_width = img.size[0]
    img_height = img.size[1]
    #image_data = img.tobytes("raw", "RGB", 0, -1)
    image_data = img.convert("RGBA").tobytes("raw", "RGBA",0,-1)

    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, img_width, img_height, 0, GL_RGBA, GL_UNSIGNED_BYTE, image_data)

### A lista abaixo armazena todos os vertices carregados dos arquivos

In [208]:
vertices_list = []    
textures_coord_list = []

### Vamos carregar cada modelo e definir funções para desenhá-los

In [209]:
def load_obj(objFile):
    modelo = load_model_from_file(objFile)
    
    verticeInicial = len(vertices_list)
    print('Processando modelo arvore.obj. Vertice inicial:',len(vertices_list))
    faces_visited = []
    for face in modelo['faces']:
        if face[2] not in faces_visited:
            print(face[2],' vertice inicial =',len(vertices_list))
            faces_visited.append(face[2])
        for vertice_id in face[0]:
            vertices_list.append( modelo['vertices'][vertice_id-1] )
        for texture_id in face[1]:
            textures_coord_list.append( modelo['texture'][texture_id-1] )
    print('Processando modelo arvore.obj. Vertice final:',len(vertices_list))
    verticeFinal = len(vertices_list)
    
    return verticeInicial, verticeFinal - verticeInicial

In [210]:
verticeInicialPessoa, quantosVerticesPessoa = load_obj('objetos/pessoa/pessoa.obj')

load_texture_from_file(0, 'objetos/pessoa/pessoa.png')

Processando modelo arvore.obj. Vertice inicial: 0
1m4g3n-texura-An1we.png  vertice inicial = 0
Borde  vertice inicial = 3846
Processando modelo arvore.obj. Vertice final: 21330
objetos/pessoa/pessoa.png RGBA


In [211]:
verticeInicialLoja, quantosVerticesLoja = load_obj('objetos/loja/loja.obj')

load_texture_from_file(1, 'objetos/loja/loja.jpg')

Processando modelo arvore.obj. Vertice inicial: 21330
Material  vertice inicial = 21330
10065_Corner_Grocery_Store_V2  vertice inicial = 21354
Processando modelo arvore.obj. Vertice final: 112602
objetos/loja/loja.jpg RGB


In [212]:
verticeInicialCarro, quantosVerticesCarro = load_obj('objetos/carro/carro.obj')

load_texture_from_file(2, 'objetos/carro/carro.png')

Processando modelo arvore.obj. Vertice inicial: 112602
Material.car2  vertice inicial = 112602
Processando modelo arvore.obj. Vertice final: 123996
objetos/carro/carro.png RGB


In [213]:
verticeInicialPolicia, quantosVerticesPolicia = load_obj('objetos/policia/policia.obj')

load_texture_from_file(3, 'objetos/policia/policia.png')

Processando modelo arvore.obj. Vertice inicial: 123996
Material.car9  vertice inicial = 123996
Processando modelo arvore.obj. Vertice final: 139722
objetos/policia/policia.png RGB


In [214]:
verticeInicialMesa, quantosVerticesMesa = load_obj('objetos/mesa/mesa.obj')

load_texture_from_file(4, 'objetos/mesa/mesa.png')

Processando modelo arvore.obj. Vertice inicial: 139722
Material  vertice inicial = 139722
Processando modelo arvore.obj. Vertice final: 148476
objetos/mesa/mesa.png RGBA


In [215]:
verticeInicialChao, quantosVerticesChao = load_obj('objetos/chao/chao.obj')

load_texture_from_file(5, 'objetos/chao/chao.png')

Processando modelo arvore.obj. Vertice inicial: 148476
Material.001_BrickRound0108_5_S.jpg  vertice inicial = 148476
Processando modelo arvore.obj. Vertice final: 246780
objetos/chao/chao.png RGB


In [216]:
verticeInicialBalao, quantosVerticesBalao = load_obj('objetos/balao/balao.obj')

load_texture_from_file(6, 'objetos/balao/balao.png')

Processando modelo arvore.obj. Vertice inicial: 246780
WhitePlastic  vertice inicial = 246780
BalloonHeart  vertice inicial = 251028
Processando modelo arvore.obj. Vertice final: 489876
objetos/balao/balao.png RGBA


In [217]:
verticeInicialCadeira, quantosVerticesCadeira = load_obj('objetos/cadeira/cadeira.obj')

load_texture_from_file(7, 'objetos/cadeira/cadeira.png')

Processando modelo arvore.obj. Vertice inicial: 489876
MATERIAL_CHAIR_01_  vertice inicial = 489876
Processando modelo arvore.obj. Vertice final: 490596
objetos/cadeira/cadeira.png RGB


In [218]:
verticeInicialCeu, quantosVerticesCeu = load_obj('objetos/ceu/ceu.obj')

load_texture_from_file(8, 'objetos/ceu/ceu.png')

Processando modelo arvore.obj. Vertice inicial: 490596
surfaceShader1SG  vertice inicial = 490596
Processando modelo arvore.obj. Vertice final: 492876
objetos/ceu/ceu.png RGBA


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Nós agora vamos requisitar dois slots.
* Um para enviar coordenadas dos vértices.
* Outros para enviar coordenadas de texturas.

In [219]:
# Request a buffer slot from GPU
buffer = glGenBuffers(2)


###  Enviando coordenadas de vértices para a GPU

In [220]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)


###  Enviando coordenadas de textura para a GPU

In [221]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)





### Desenhando nossos modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo
* É necessário indicar qual o ID da textura do modelo


In [222]:
def desenha_pessoa(t_y):
    
    
    # aplica a matriz model
    
    # rotacao
    angle = 180.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x = -8.0; t_z = 11;
    
    # escala
    s_x = 0.45; s_y = 0.45; s_z = 0.45;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    
    ### desenho o tronco da arvore
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 0)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, verticeInicialPessoa, quantosVerticesPessoa) ## renderizando
    



In [223]:
def desenha_loja():
    
    
    # aplica a matriz model
    
    # rotacao
    angle = 180.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x = 0.0; t_y = 0.0; t_z = 15.0;
    
    # escala
    s_x = 0.015; s_y = 0.015; s_z = 0.015;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)

    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 1)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, verticeInicialLoja, quantosVerticesLoja) ## renderizando
    

In [224]:
def desenha_carro(angle=180, t_x=8.0, t_z=15.0):
    
    
    # aplica a matriz model
    
    # rotacao
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_y = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)

    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 2)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, verticeInicialCarro, quantosVerticesCarro) ## renderizando

In [225]:
def desenha_policia():
    
    
    # aplica a matriz model
    
    # rotacao
    angle = 180.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x = 8.0; t_y = 0.0; t_z = 20.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)

    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 3)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, verticeInicialPolicia, quantosVerticesPolicia) ## renderizando

In [226]:
def desenha_mesa():
    
    # aplica a matriz model
    
    # rotacao
    angle = 180.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x = -5.0; t_y = 0.0; t_z = 20.0;
    
    # escala
    s_x = 8.0; s_y = 4.0; s_z = 2.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)

    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 4)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, verticeInicialMesa, quantosVerticesMesa) ## renderizando

In [227]:
def desenha_chao(t_x=0, t_z=0):
    
    # aplica a matriz model
    
    # rotacao
    angle = 180.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_y = -0.5;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)

    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 5)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, verticeInicialChao, quantosVerticesChao) ## renderizando

In [228]:
def desenha_balao(escala=1):
    
    # aplica a matriz model
    
    # rotacao
    angle = 180.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x = -3; t_y = 2.5; t_z = 22;
    
    # escala
    s_x = escala; s_y = escala; s_z = escala;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)

    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 6)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, verticeInicialBalao, quantosVerticesBalao) ## renderizando

In [229]:
def desenha_cadeira():
    
    # aplica a matriz model
    
    # rotacao
    angle = 180.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x = -5.0; t_y = 0.0; t_z = 22.0;
    
    # escala
    s_x = 2.6; s_y = 2.6; s_z = 2.6;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)

    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 7)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, verticeInicialCadeira, quantosVerticesCadeira) ## renderizando

In [230]:
def desenha_ceu():
    
    # aplica a matriz model
    
    # rotacao
    angle = 180.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x = 0.0; t_y = 0.0; t_z = 15.0;
    
    # escala
    s_x = 1.1; s_y = 1.1; s_z = 1.1;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)

    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 8)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, verticeInicialCeu, quantosVerticesCeu) ## renderizando

### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [231]:
#cameraPos   = glm.vec3(0.0,  0.0,  1.0);
#cameraFront = glm.vec3(0.0,  0.0, -1.0);
#cameraUp    = glm.vec3(0.0,  1.0,  0.0);

# camera
cameraPos   = glm.vec3(0.0, 0.0, 3.0)
cameraFront = glm.vec3(0.0, 0.0, -1.0)
cameraUp    = glm.vec3(0.0, 1.0, 0.0)

firstMouse = True
yaw   = -90.0  # yaw is initialized to -90.0 degrees since a yaw of 0.0 results in a direction vector pointing to the right so we initially rotate a bit to the left.
pitch =  0.0
lastX =  largura / 2.0
lastY =  altura / 2.0
fov   =  45.0

# timing
deltaTime = 0.0  # time between current frame and last frame
lastFrame = 0.0


firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2


def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode
    global pulo, estacionar, aumentar

    if key == glfw.KEY_ESCAPE and action == glfw.PRESS:
        glfw.set_window_should_close(window, True)
    
    cameraSpeed = 100 * deltaTime
    if key == glfw.KEY_W and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos += cameraSpeed * cameraFront
    
    if key == glfw.KEY_S and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos -= cameraSpeed * cameraFront
    
    if key == glfw.KEY_A and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == glfw.KEY_D and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed

    cameraPos.x = glm.clamp(cameraPos.x, -20.0, 16.0)
    cameraPos.z = glm.clamp(cameraPos.z, 0.0, 27.0)
    cameraPos.y = glm.clamp(cameraPos.y, 0.5, 10.0) 

    if key == glfw.KEY_P and action == glfw.PRESS:
        polygonal_mode = not polygonal_mode

    if key == glfw.KEY_R:
        pulo = True

    if key == glfw.KEY_E:
        estacionar = True

    if key == glfw.KEY_T:
        aumentar = True
    

def framebuffer_size_callback(window, largura, altura):

    # make sure the viewport matches the new window dimensions note that width and 
    # height will be significantly larger than specified on retina displays.
    glViewport(0, 0, largura, altura)

# glfw: whenever the mouse moves, this callback is called
# -------------------------------------------------------
def mouse_callback(window, xpos, ypos):
    global cameraFront, lastX, lastY, firstMouse, yaw, pitch
   
    if (firstMouse):

        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos # reversed since y-coordinates go from bottom to top
    lastX = xpos
    lastY = ypos

    sensitivity = 0.1 # change this value to your liking
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset
    pitch += yoffset

    # make sure that when pitch is out of bounds, screen doesn't get flipped
    if (pitch > 89.0):
        pitch = 89.0
    if (pitch < -89.0):
        pitch = -89.0

    front = glm.vec3()
    front.x = glm.cos(glm.radians(yaw)) * glm.cos(glm.radians(pitch))
    front.y = glm.sin(glm.radians(pitch))
    front.z = glm.sin(glm.radians(yaw)) * glm.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)

# glfw: whenever the mouse scroll wheel scrolls, this callback is called
# ----------------------------------------------------------------------
def scroll_callback(window, xoffset, yoffset):
    global fov

    fov -= yoffset
    if (fov < 1.0):
        fov = 1.0
    if (fov > 45.0):
        fov = 45.0
    
glfw.set_key_callback(window,key_event)
glfw.set_framebuffer_size_callback(window, framebuffer_size_callback)
glfw.set_cursor_pos_callback(window, mouse_callback)
glfw.set_scroll_callback(window, scroll_callback)

# tell GLFW to capture our mouse
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)


### Matrizes Model, View e Projection

In [232]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade

    
    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform).T # pegando a transposta da matriz (glm trabalha com ela invertida)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura, inc_fov, inc_near, inc_far
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Nesse momento, nós exibimos a janela!


In [233]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

### Loop principal da janela.

In [234]:
glEnable(GL_DEPTH_TEST) ### importante para 3D
   
pulo = False
estacionar = False
aumentar = False
pessoa_t_y = 0.0
incr_pulo = 0.003
carro_angulo=180; carro_t_x=8.0; carro_t_z=15.0; incr_carro = 0.06
balao_tamanho = 2.0; incr_balao = 0.01
polygonal_mode = False

while not glfw.window_should_close(window):

    currentFrame = glfw.get_time()
    deltaTime = currentFrame - lastFrame
    lastFrame = currentFrame

    glfw.poll_events() 

    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if pulo == True:
        if pessoa_t_y >= 1.2:
            incr_pulo *= -1
        pessoa_t_y += incr_pulo
        if pessoa_t_y <= 0.0:
            incr_pulo *= -1
            pulo = False
    desenha_pessoa(pessoa_t_y)
    desenha_loja()
    if estacionar == True:
        carro_angulo += incr_carro
        carro_t_x += incr_carro/10 * -math.cos(math.radians(carro_angulo))
        carro_t_z += incr_carro/10 * math.sin(math.radians(carro_angulo))
        if carro_angulo >= 270 or carro_angulo <= 180:
            incr_carro *= -1
            estacionar = False

    desenha_carro(carro_angulo, carro_t_x, carro_t_z)
    desenha_policia()
    desenha_mesa()

    for chao_t_x in range(-20, 20, 9):
        for chao_t_z in range(0, 30, 9):
            desenha_chao(chao_t_x, chao_t_z)
    
    if aumentar == True:
        balao_tamanho += incr_balao
        if balao_tamanho <= 2.0 or balao_tamanho >= 3:
            incr_balao *= -1
            aumentar = False

    desenha_balao(balao_tamanho)
    desenha_cadeira()
    desenha_ceu()

    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)    

    glfw.swap_buffers(window)

glfw.terminate()